In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import os
n_jobs = 16
os.environ["OMP_NUM_THREADS"] = str(n_jobs)
import joblib
import click
import json
import time
from glob import glob
import itertools
import collections.abc
import sys
from tqdm.notebook import tqdm
# !{sys.executable} -m pip install qcircuit
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import re
from joblib import Parallel, delayed, dump, load


In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12
 
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [ ]:
top=os.getcwd()
with open(os.path.join(top,'5_DDCC_train.bin'),'rb') as f:
    ddcc_train = joblib.load(f)

with open(os.path.join(top,'5_DDCC_test.bin'),'rb') as f:
    ddcc_test = joblib.load(f)

with open(os.path.join(top,'5_DDCC_scaler.bin'),'rb') as f:
    ddcc_scaler = joblib.load(f)

X_ddcc_train, y_ddcc_train = ddcc_train['X'],ddcc_train['y']
X_ddcc_test, y_ddcc_test = ddcc_test['X'],ddcc_test['y']

X_ddcc_train = X_ddcc_train.reshape(-1,64,5)[0:4]
X_ddcc_test = X_ddcc_test.reshape(-1,64,5)[0:4]
y_ddcc_train = y_ddcc_train.reshape(-1,64)[0:4]
y_ddcc_test = y_ddcc_test.reshape(-1,64)[0:4]


# X_train, y_train = X_ddcc_train, y_ddcc_train
# X_test, y_test = X_ddcc_test, y_ddcc_test
X_train, y_train = X_ddcc_train, y_ddcc_train
X_test, y_test = X_ddcc_test, y_ddcc_test

# X_train = [X_train[i:i+4] for i in range(0,len(X_train),4)]
# X_test = [X_test[i:i+4] for i in range(0,len(X_test),4)]
scaler = ddcc_scaler

# print(len(X_train),X_train[0].shape,X_train[-1].shape)
print(y_train.shape, y_test.shape)

In [ ]:
def grab_data(path):
    '''
    Given a globbed path, return the job_n.txt, *_train.txt, and *_test.txt files
    '''
    iterations = []
    train_metric = []
    test_metric = []
    for i in path:
        try:
            numbers = int(re.search(r'jobs_(\d+)\.txt', i).group(1))
            iterations.append(i)
        except:
            if 'test' in i:
                test_metric.append(i)
            else:
                train_metric.append(i)
            
    return sorted(iterations,key=lambda x: int(re.search(r'jobs_(\d+)\.txt', x).group(1))), train_metric, test_metric

In [ ]:
def fetch_jobs_from_file(service, filename):
    """Fetch all job results from a given file."""
    with open(filename, 'r') as f:
        job_ids = f.readlines()
    return [service.job(job_id.strip()).result() for job_id in job_ids]

def grab_jobs(lst):
    """Parallelize over job files, fetching results concurrently."""
    service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance='pinq-quebec-hub/univ-toronto/default'
    )

    # Parallel processing of files
    jobs = Parallel(n_jobs=-1,backend='threading')(
        delayed(fetch_jobs_from_file)(service, filename) for filename in tqdm(lst)
    )

    return np.array(jobs)

In [ ]:
iter_100, train_100, test_100 = grab_data(glob('100_iter/*txt'))
iter_500, train_500, test_500 = grab_data(glob('500_iter/*txt'))
iter_1000, train_1000, test_1000 = grab_data(glob('1000_iter/*txt'))

In [ ]:
# results_100 = grab_jobs(iter_100)
# y_100 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_100]).reshape(*results_100.shape)


# results_500 = grab_jobs(iter_500)
# y_500 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_500]).reshape(*results_500.shape)

# results_1000 = grab_jobs(iter_1000)
# y_1000 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_1000]).reshape(*results_1000.shape)
# with open('y_100.npy', 'wb') as f:
#     np.save(f, y_100)
# with open('y_500.npy', 'wb') as f:
#     np.save(f, y_500)
# with open('y_1000.npy', 'wb') as f:
#     np.save(f, y_1000)

# results_train_100 = grab_jobs(train_100)
# y_100_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_100]).reshape(*y_train.shape)


# results_train_500 = grab_jobs(train_500)
# y_500_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_500]).reshape(*y_train.shape)

# results_train_1000 = grab_jobs(train_1000)
# y_1000_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_1000]).reshape(*y_train.shape)

# results_test_100 = grab_jobs(test_100)
# y_100_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_100]).reshape(*y_test.shape)


# results_test_500 = grab_jobs(test_500)
# y_500_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_500]).reshape(*y_test.shape)

# results_test_1000 = grab_jobs(test_1000)
# y_1000_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_1000]).reshape(*y_test.shape)

# with open('y_100_test.npy', 'wb') as f:
#     np.save(f, y_100_test)
# with open('y_500_test.npy', 'wb') as f:
#     np.save(f, y_500_test)
# with open('y_1000_test.npy', 'wb') as f:
#     np.save(f, y_1000_test)


# with open('y_100_train.npy', 'wb') as f:
#     np.save(f, y_100_train)
# with open('y_500_train.npy', 'wb') as f:
#     np.save(f, y_500_train)
# with open('y_1000_train.npy', 'wb') as f:
#     np.save(f, y_1000_train)

In [ ]:
scaler.inverse_transform

In [ ]:


with open('y_100_test.npy', 'rb') as f:
    y_100_test = np.load(f)
with open('y_500_test.npy', 'rb') as f:
    y_500_test = np.load(f)
with open('y_1000_test.npy', 'rb') as f:
    y_1000_test = np.load(f)   


    
with open('y_100_train.npy', 'rb') as f:
    y_100_train = np.load(f)
with open('y_500_train.npy', 'rb') as f:
    y_500_train = np.load(f)
with open('y_1000_train.npy', 'rb') as f:
    y_1000_train = np.load(f)       
    
with open('y_100.npy', 'rb') as f:
    y_100 = np.load(f)
with open('y_500.npy', 'rb') as f:
    y_500 = np.load(f)
with open('y_1000.npy', 'rb') as f:
    y_1000 = np.load(f)    
    


In [ ]:
y_100_test = scaler.inverse_transform(y_100_test.flatten().reshape(-1,1)).flatten()
y_500_test = scaler.inverse_transform(y_500_test.flatten().reshape(-1,1)).flatten()
y_1000_test = scaler.inverse_transform(y_1000_test.flatten().reshape(-1,1)).flatten()
y_100_train = scaler.inverse_transform(y_100_train.flatten().reshape(-1,1)).flatten()
y_500_train = scaler.inverse_transform(y_500_train.flatten().reshape(-1,1)).flatten()
y_1000_train = scaler.inverse_transform(y_1000_train.flatten().reshape(-1,1)).flatten()

In [ ]:

y_100 = scaler.inverse_transform(y_100.flatten().reshape(-1,1)).reshape(*y_100.shape)
y_500 = scaler.inverse_transform(y_500.flatten().reshape(-1,1)).reshape(*y_500.shape)
y_1000 = scaler.inverse_transform(y_1000.flatten().reshape(-1,1)).reshape(*y_1000.shape)



y_ddcc_train = scaler.inverse_transform(y_ddcc_train.flatten().reshape(-1,1)).reshape(*y_ddcc_train.shape)
y_ddcc_test = scaler.inverse_transform(y_ddcc_test.flatten().reshape(-1,1)).reshape(*y_ddcc_test.shape)

In [ ]:
r2_score(y_ddcc_train.flatten(),y_100_train.flatten())

In [ ]:
r2_score(y_ddcc_train.flatten(),y_500_train.flatten())

In [ ]:
r2_score(y_ddcc_train.flatten(),y_1000_train.flatten())

In [ ]:
MSE_100 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_100]
MSE_500 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_500]
MSE_1000 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_1000]

combined_MSE = MSE_100+MSE_500+MSE_1000
min_MSE_idx = np.argmin(MSE_100+MSE_500+MSE_1000)

In [ ]:
best_train = np.vstack([y_100,y_500,y_1000])[min_MSE_idx]

In [ ]:
with open(os.path.join(os.path.expanduser('~'),'qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_results.json')) as f:
    statevector = json.load(f)

In [ ]:
'/Users/grierjones/qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_predicted_values.csv'

In [ ]:
statevectordf = pd.read_csv(os.path.join(os.path.expanduser('~'),'qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_predicted_values.csv'))
statevectordf['Predicted'] = [float(i.strip('[]')) for i in statevectordf['Predicted']]
statevectordf['Reference'] = [float(i.strip('[]')) for i in statevectordf['Reference']]
statevectordf['Device'] = len(statevectordf)*['State Vector']



In [ ]:
statevectordf

In [ ]:
dfbest = pd.DataFrame(np.vstack([y_ddcc_train.flatten(),best_train.flatten()]),index=['Reference','Predicted']).T
dfbest['Device'] = len(dfbest)*['ibm_quebec']
dfbest['Data'] = len(dfbest)*['Train']

In [ ]:
sns.scatterplot(data=pd.concat([statevectordf,dfbest]),x='Reference',y='Predicted',hue='Device',style='Data')

In [ ]:
statevectordf.max()

In [ ]:
spread = 10e-2
# Create the figure with a 2D grid (scatter + KDE for Predicted + KDE for Reference)
fig, axes = plt.subplots(
    2, 2, 
    figsize=(8, 8),
    gridspec_kw={'width_ratios': [4, 1], 'height_ratios': [1, 4]},
    constrained_layout=True,
    # sharey='row',  # Keep y-axis sharing, but remove sharex to control ticks manually,
    # sharex='col'
)
axes[0, -1].axis("off")
sns.scatterplot(data=dfbest,x='Reference',y='Predicted',label="Best ibm_quebec: R$^{2}=$"+f"{r2_score(dfbest['Reference'],dfbest['Predicted']):.4f}", edgecolors='black',ax=axes[1,0])
# sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector: R$^{2}=$"+f"{r2_score(statevectordf['Reference'],statevectordf['Predicted']):.4f}", edgecolors='black',ax=axes[1,0])
axes[1,0].plot(range(-1,2),range(-1,2),'k--')
axes[1,0].set_ylim(-spread,spread)
axes[1,0].set_xlim(-spread,spread)
axes[1,0].set_ylabel("Predicted t$_{2}$-amplitudes")
axes[1,0].set_xlabel("Calculated t$_{2}$-amplitudes")
axes[1,0].legend()


# sns.kdeplot(data=pd.concat([dfbest,statevectordf[statevectordf['Data']=='Train']]),hue='Device',x='Reference',ax=axes[0,0],fill=True)
sns.kdeplot(data=dfbest,x='Reference',ax=axes[0,0],fill=True, bw_adjust=2)
# sns.kdeplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',ax=axes[0,0],fill=True, bw_adjust=2)
axes[0,0].set_xticklabels([])  # Hide labels but keep ticks
axes[0,0].set_xlabel("")  # Remove x-labels
axes[0,0].set_xlim(-spread,spread)
axes[0,0].set_ylim(0,100)

# sns.kdeplot(data=pd.concat([dfbest,statevectordf[statevectordf['Data']=='Train']]),hue='Device',y='Predicted',ax=axes[1,1],fill=True)
sns.kdeplot(data=dfbest,y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2)
# sns.kdeplot(data=statevectordf[statevectordf['Data']=='Train'],y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2)
axes[1,1].set_yticklabels([])  # Hide labels but keep ticks
axes[1,1].set_ylabel("")  # Remove x-labels
axes[1,1].set_ylim(-spread,spread)
axes[1,1].set_xlim(0,100)
# plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust width and height spacing
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/bestibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# # plt.scatter(y_ddcc_train.flatten(),best_train.flatten(),label="Best ibm_quebec R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),best_train.flatten()):.4f}",color='r', edgecolors='black')
# sns.scatterplot(data=dfbest,x='Reference',y='Predicted',label="Best ibm_quebec: R$^{2}=$"+f"{r2_score(dfbest['Reference'],dfbest['Predicted']):.4f}", edgecolors='black')
# sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector: R$^{2}=$"+f"{r2_score(statevectordf['Reference'],statevectordf['Predicted']):.4f}", edgecolors='black')
# plt.plot(range(-1,2),range(-1,2),'k--')
# plt.ylim(-3e-2,3e-2)
# plt.xlim(-3e-2,3e-2)
# plt.ylabel("Predicted t$_{2}$-amplitudes")
# plt.xlabel("Calculated t$_{2}$-amplitudes")
# plt.legend()
# plt.tight_layout()
# plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/bestibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
plt.scatter(y_ddcc_train.flatten(),y_1000_train.flatten(),label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_train.flatten()):.4f}",color='b', edgecolors='black')
plt.scatter(y_ddcc_test.flatten(),y_1000_test.flatten(),label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_test.flatten()):.4f}",color='g', edgecolors='black')
sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector Train: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data']=='Train']['Reference'],statevectordf[statevectordf['Data']=='Train']['Predicted']):.4f}", edgecolors='black')
sns.scatterplot(data=statevectordf[statevectordf['Data']=='Test'],x='Reference',y='Predicted',label="State Vector Test: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data']=='Test']['Reference'],statevectordf[statevectordf['Data']=='Test']['Predicted']):.4f}", edgecolors='black')
plt.plot(range(-1,2),range(-1,2),'k--')
plt.ylim(-3e-2,3e-2)
plt.xlim(-3e-2,3e-2)
plt.ylabel("Predicted t$_{2}$-amplitudes")
plt.xlabel("Calculated t$_{2}$-amplitudes")
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/finalibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
finaldf = pd.DataFrame(np.hstack([np.vstack([y_ddcc_train.flatten(),y_1000_train.flatten()]),np.vstack([y_ddcc_test.flatten(),y_1000_test.flatten()])]),index=['Reference','Predicted']).T
finaldf['Data'] = len(y_ddcc_train.flatten())*['Train']+len(y_ddcc_test.flatten())*['Test']
finaldf['Device'] = len(finaldf)*['ibm_quebec']

In [ ]:
y_ddcc_train.shape

In [ ]:
cmap = sns.color_palette('Paired',6)

In [ ]:
cmap

In [ ]:
finaldf["Data"] = finaldf["Data"] + " (" + finaldf["Device"] + ")"
statevectordf["Data"] = statevectordf["Data"] + " (" + statevectordf["Device"] + ")"


In [ ]:
statevectordf['Reference']

In [ ]:
finaldf

In [ ]:
y_1000_train.flatten()

In [ ]:
spread = 10e-2
# Create the figure with a 2D grid (scatter + KDE for Predicted + KDE for Reference)
fig, axes = plt.subplots(
    2, 2, 
    figsize=(10, 10),
    gridspec_kw={'width_ratios': [4, 1.7], 'height_ratios': [1.7, 4]},
    constrained_layout=True,
    # sharey='row',  # Keep y-axis sharing, but remove sharex to control ticks manually,
    # sharex='col'
)
axes[0, -1].axis("off")
# sns.scatterplot(data=finaldf,x='Reference',y='Predicted',hue='Data',style='Data',markers=['o', 's'], edgecolors='black',ax=axes[1,0],palette=)
axes[1,0].scatter(y_ddcc_train.flatten(),y_1000_train.flatten(),marker='s',label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_train.flatten()):.4f}",color=cmap[0], edgecolors='black')
axes[1,0].scatter(y_ddcc_test.flatten(),y_1000_test.flatten(),label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_test.flatten()):.4f}",color=cmap[1], edgecolors='black')
# sns.scatterplot(data=statevectordf,hue='Data',x='Reference',y='Predicted',style='Data',markers=['d', 'D'], edgecolors='black',ax=axes[1,0],palette=cmap[2:4])
axes[1,0].plot(range(-1,2),range(-1,2),'k--')
axes[1,0].set_ylim(-spread,spread)
axes[1,0].set_xlim(-spread,spread)
axes[1,0].set_ylabel("Predicted t$_{2}$-amplitudes")
axes[1,0].set_xlabel("Calculated t$_{2}$-amplitudes")
axes[1,0].legend()


# sns.histplot(data=pd.concat([finaldf,statevectordf]),hue='Data',x='Reference',ax=axes[0,0],fill=True,palette=cmap[0:4])
# sns.histplot(data=finaldf,hue='Data',x='Reference',ax=axes[0,0],fill=True,palette=cmap[0:2],stat='probability',kde=True)
# axes[0,0].set_yscale('log')
sns.kdeplot(data=finaldf,hue='Data',x='Reference',ax=axes[0,0],fill=True, bw_adjust=2,palette=cmap[0:2])
# sns.kdeplot(data=statevectordf,hue='Data',x='Reference',ax=axes[0,0],fill=True, bw_adjust=2,palette=cmap[3:5])
axes[0,0].set_xticklabels([])  # Hide labels but keep ticks
axes[0,0].set_xlabel("")  # Remove x-labels
axes[0,0].set_xlim(-spread,spread)
axes[0,0].set_ylim(1,60)
# axes[0,0].legend(loc=3)

# sns.histplot(data=pd.concat([finaldf,statevectordf]),hue='Data',y='Predicted',ax=axes[1,1],fill=True,palette=cmap[0:4])
# sns.histplot(data=finaldf,hue='Data',y='Predicted',ax=axes[1,1],fill=True,palette=cmap[0:2],stat='probability',kde=True)
# axes[1,1].set_xscale('log')
sns.kdeplot(data=finaldf,hue='Data',y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2,palette=cmap[0:2])
# sns.kdeplot(data=statevectordf,hue='Data',y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2,palette=cmap[3:5])
# axes[1,1].set_xticks(np.hstack([0,np.logspace(0,4,3)]))
# axes[1,1].set_xticklabels(['0']+["10$^{"+f"{np.log10(i):n}"+"}$" for i in np.logspace(0,4,3)])  # Hide labels but keep ticks
axes[1,1].set_yticklabels([])  # Hide labels but keep ticks
axes[1,1].set_ylabel("")  # Remove x-labels
axes[1,1].set_ylim(-spread,spread)
axes[1,1].set_xlim(1,60)
plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust width and height spacing
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/finalibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
len([len(y_ddcc_train.flatten())*['Train']]+[len(y_ddcc_test.flatten())*['Test']])

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from sklearn.metrics import r2_score

# fig, ax = plt.subplots(figsize=(8, 6))

# # Scatter plots
# ax.scatter(y_ddcc_train.flatten(), y_1000_train.flatten(), 
#            label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(), y_1000_train.flatten()):.4f}", 
#            color='b', edgecolors='black')
# ax.scatter(y_ddcc_test.flatten(), y_1000_test.flatten(), 
#            label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_test.flatten(), y_1000_test.flatten()):.4f}", 
#            color='g', edgecolors='black')

# sns.scatterplot(data=statevectordf[statevectordf['Data'] == 'Train'], x='Reference', y='Predicted', 
#                 label="State Vector Train: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data'] == 'Train']['Reference'], statevectordf[statevectordf['Data'] == 'Train']['Predicted']):.4f}", 
#                 ax=ax, edgecolor='black')

# sns.scatterplot(data=statevectordf[statevectordf['Data'] == 'Test'], x='Reference', y='Predicted', 
#                 label="State Vector Test: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data'] == 'Test']['Reference'], statevectordf[statevectordf['Data'] == 'Test']['Predicted']):.4f}", 
#                 ax=ax, edgecolor='black')

# # Identity line
# ax.plot(np.linspace(-1, 2, 100), np.linspace(-1, 2, 100), 'k--')

# ax.set_ylim(-3e-2, 3e-2)
# ax.set_xlim(-3e-2, 3e-2)
# ax.set_ylabel("Predicted t$_{2}$-amplitudes")
# ax.set_xlabel("Calculated t$_{2}$-amplitudes")
# ax.legend()
# plt.tight_layout()

# # Create twin axes for KDE plots
# top_ax = ax.twiny()
# right_ax = ax.twinx()

# # KDE distributions
# sns.kdeplot(statevectordf['Reference'], ax=top_ax, color='gray', lw=2, clip=(-3e-2, 3e-2))
# sns.kdeplot(statevectordf['Predicted'], ax=right_ax, color='gray', lw=2, clip=(-3e-2, 3e-2))

# # Hide tick labels for KDE axes
# top_ax.set_xticks([])
# right_ax.set_yticks([])

# plt.show()


In [ ]:
plt.scatter(min_MSE_idx,combined_MSE[min_MSE_idx],color='r',label=f'MSE:{combined_MSE[min_MSE_idx]:.4e}')
plt.plot(MSE_100+MSE_500+MSE_1000)
plt.ylabel('Training Loss (MSE)')
plt.xlabel('Iterations')
plt.xlim(-10,900)
plt.ylim(0,8e-4)
# plt.hlines(statevector['MSE_train'][0],-100,1e4,color='r',linestyle='--',label=f'State Vector MSE:{statevector['MSE_train'][0]:.4e}')
# plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/ibmq_loss.png'),dpi=300,bbox_inches='tight')
plt.show()